## 模型导出

使用LoRA 适配器训练好模型后，每次推理时候都需要分别加载基座模型和 LoRA 适配器，一方面比较繁琐，另一方面也不利于模型部署，因此有必要将基座模型和 LoRA 适配器进行合并，导出成一个模型。

LLamaFactory中自带模型合并的功能，只需要从`examples/merge_lora/llama3_lora_sft.yaml`拷贝一份配置根据自己的情况进行修改。修改后的配置文件示例如下：

In [4]:
!cat /data2/downloads/LLaMA-Factory/merge_qwen2_lora_sft.yaml

### Note: DO NOT use quantized model or quantization_bit when merging lora adapters

### model
model_name_or_path: /data2/anti_fraud/models/modelscope/hub/Qwen/Qwen2-1___5B-Instruct
adapter_name_or_path: /data2/anti_fraud/models/Qwen2-1___5B-Instruct_ft_0826/checkpoint-1400
template: qwen
finetuning_type: lora

### export
export_dir: /data2/anti_fraud/models/Qwen2-1__5B-Instruct-anti_fraud_1__0
export_size: 2
export_device: cuda
export_legacy_format: false


- export_size: 指定导出的模型文件分片大小，单位为GB, 适用于模型比较大需要分片导出的场景，以便在内存限制的设备上加载。
- export_device: 导出模型时使用的设备。可以选择 cpu 或 cuda（即 GPU）。如果有强大的 GPU 可以使用，选择 cuda 可以加快导出速度。
- export_legacy_format: 指定是否使用旧格式导出模型。通常情况下，选择 false 以导出使用最新格式的模型。

In [ ]:
使用`llamafactory-cli`执行export命令导出模型。

In [1]:
!llamafactory-cli export /data2/downloads/LLaMA-Factory/merge_qwen2_lora_sft.yaml

[2024-08-29 11:25:38,878] [INFO] [real_accelerator.py:203:get_accelerator] Setting ds_accelerator to cuda (auto detect)
 [WARNING]  async_io requires the dev libaio .so object and headers but these were not found.
 [WARNING]  async_io: please install the libaio-dev package with apt
 [WARNING]  If libaio is already installed (perhaps from source), try setting the CFLAGS and LDFLAGS environment variables to where it can be found.
 [WARNING]  Please specify the CUTLASS repo directory as environment variable $CUTLASS_PATH
 [WARNING]  sparse_attn requires a torch version >= 1.5 and < 2.0 but detected 2.4
 [WARNING]  using untested triton version (3.0.0), only 1.0.0 is known to be compatible
/data2/anaconda3/envs/python3_10/lib/python3.10/site-packages/deepspeed/runtime/zero/linear.py:49: FutureWarning: `torch.cuda.amp.custom_fwd(args...)` is deprecated. Please use `torch.amp.custom_fwd(args..., device_type='cuda')` instead.
  def forward(ctx, input, weight, bias=None):
/data2/anaconda3/envs

In [ ]:
查看导出的模型文件，可以看到已经按照我们的配置将模型参数进行了两个分片。

In [3]:
!ls -l /data2/anti_fraud/models/Qwen2-1__5B-Instruct-anti_fraud_1__0

total 3026368
-rw-rw-r-- 1 xiaoguanghua xiaoguanghua         80 Aug 29 11:30 added_tokens.json
-rw-rw-r-- 1 xiaoguanghua xiaoguanghua        748 Aug 29 11:30 config.json
-rw-rw-r-- 1 xiaoguanghua xiaoguanghua        242 Aug 29 11:30 generation_config.json
-rw-rw-r-- 1 xiaoguanghua xiaoguanghua    1671853 Aug 29 11:30 merges.txt
-rw-rw-r-- 1 xiaoguanghua xiaoguanghua 1975314632 Aug 29 11:30 model-00001-of-00002.safetensors
-rw-rw-r-- 1 xiaoguanghua xiaoguanghua 1112152304 Aug 29 11:30 model-00002-of-00002.safetensors
-rw-rw-r-- 1 xiaoguanghua xiaoguanghua      27693 Aug 29 11:30 model.safetensors.index.json
-rw-rw-r-- 1 xiaoguanghua xiaoguanghua        367 Aug 29 11:30 special_tokens_map.json
-rw-rw-r-- 1 xiaoguanghua xiaoguanghua       1532 Aug 29 11:30 tokenizer_config.json
-rw-rw-r-- 1 xiaoguanghua xiaoguanghua    7028043 Aug 29 11:30 tokenizer.json
-rw-rw-r-- 1 xiaoguanghua xiaoguanghua    2776833 Aug 29 11:30 vocab.json


In [5]:
!ls -l /data2/anti_fraud/models/modelscope/hub/Qwen/Qwen2-1___5B-Instruct

total 3026356
-rw-rw-r-- 1 xiaoguanghua xiaoguanghua        660 Aug  1 15:58 config.json
-rw-rw-r-- 1 xiaoguanghua xiaoguanghua         48 Aug  1 15:58 configuration.json
-rw-rw-r-- 1 xiaoguanghua xiaoguanghua        242 Aug  1 15:58 generation_config.json
-rw-rw-r-- 1 xiaoguanghua xiaoguanghua      11344 Aug  1 15:58 LICENSE
-rw-rw-r-- 1 xiaoguanghua xiaoguanghua    1671839 Aug  1 15:58 merges.txt
-rw-rw-r-- 1 xiaoguanghua xiaoguanghua 3087467144 Aug  1 16:16 model.safetensors
-rw-rw-r-- 1 xiaoguanghua xiaoguanghua       3551 Aug  1 16:17 README.md
-rw-rw-r-- 1 xiaoguanghua xiaoguanghua       1287 Aug  1 16:17 tokenizer_config.json
-rw-rw-r-- 1 xiaoguanghua xiaoguanghua    7028015 Aug  1 16:17 tokenizer.json
-rw-rw-r-- 1 xiaoguanghua xiaoguanghua    2776833 Aug  1 16:17 vocab.json


## 测试模型性能

In [ ]:
%run evaluate.py
model_path = '/data2/anti_fraud/models/Qwen2-1__5B-Instruct-anti_fraud_1__0'

device = 'cuda'

In [10]:
testdata_path = '/data2/anti_fraud/dataset/eval0819.jsonl'
evaluate(model_path, '', testdata_path, device, batch=True, debug=True)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

run in batch mode, batch_size=8


progress: 100%|██████████| 2348/2348 [01:58<00:00, 19.85it/s]

tn：1162, fp:3, fn:70, tp:1113
precision: 0.9973118279569892, recall: 0.9408284023668639


In [9]:

testdata_path = '/data2/anti_fraud/dataset/test0819.jsonl'
evaluate(model_path, '', testdata_path, device, batch=True, debug=True)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

run in batch mode, batch_size=8


progress: 100%|██████████| 2349/2349 [01:56<00:00, 20.13it/s]

tn：1136, fp:31, fn:162, tp:1020
precision: 0.9705042816365367, recall: 0.8629441624365483


## 模型部署

查看配置文件：

In [ ]:
!less /data2/downloads/LLaMA-Factory/qwen2_inference.yaml

model_name_or_path: /data2/anti_fraud/models/Qwen2-1__5B-Instruct-anti_fraud_1__0
template: qwen
infer_backend: vllm
vllm_enforce_eager: true
/data2/downloads/LLaMA-Factory/qwen2_inference.yaml (END)

通过llamafactory启动推理服务的方式：
```
llamafactory-cli api /data2/downloads/LLaMA-Factory/qwen2_inference.yaml
```
为了收集日志及指定环境变量，封装为一个启动脚本。

In [ ]:
!less ~/anti_fraud_start.sh

启动服务后，使用openai的api来测试服务功能。

## 推理测试

In [ ]:
%%time

# api_call_example.py
from openai import OpenAI

def predict(prompt):
    client = OpenAI(api_key="0",base_url="http://192.168.31.200:8001/v1")
    messages = [{"role": "user", "content": prompt}]
    result = client.chat.completions.create(messages=messages, model="Qwen2-1__5B-Instruct")
    return result.choices[0].message.content

predict("详细介绍下你自己。")

In [6]:
def build_fraud_prompt(content):
    return f"下面是一段对话文本, 请分析对话内容是否有诈骗风险，以json格式输出你的判断结果(is_fraud: true\/false)。\n{content}"

In [7]:
%%time
content = '小明: 我们有专业的团队进行风险管理，可以确保你的投资安全。而且我们会提供实时的投资动态，让你随时掌握投资情况。\n小红: 那我什么时候能回收投资并获得收益呢？\n小明: 投资期限为1年，到期后你就可以回收本金和收益。\n小红: 听起来还不错，我会考虑一下。谢谢你的介绍。\n小明: 不客气，如果你有任何问题，随时可以问我。希望你能抓住这个机会，获得更多的财富。'
predict(build_fraud_prompt(content))

CPU times: user 53.3 ms, sys: 3.71 ms, total: 57 ms
Wall time: 217 ms


'{"is_fraud": true}'

In [8]:
%%time
content = '发言人3: 然后从不管是分业务板块的一个营收占比和分地区板块的或分地区的一个营收占比来看的话，首先这个屠宰基本上是占了公司总体营收的90%以上的一个比例，但是我们可以看到左下角的这个图，近几年畜禽养殖板块其实更多就来自于生猪养殖板块，它对于整个营收的一个占比它其实有一个明显的提升，而且随着未来公司出栏量的一个增长，包括可能猪价的一个后面有一个周期的一个反转的话，可能未来养殖板块它占总营收的一个比例仍然会有一个快速的一个提升。'
predict(build_fraud_prompt(content))

CPU times: user 52.9 ms, sys: 3.34 ms, total: 56.2 ms
Wall time: 215 ms


'{"is_fraud": false}'